In [2]:
from skimage.measure import marching_cubes
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

In [3]:
import os
import numpy as np
import cv2
import nibabel as nib
#%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv3D, MaxPooling3D, UpSampling3D, concatenate
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from skimage.transform import resize
from mpl_toolkits.mplot3d import Axes3D
import tensorflow as tf
import scipy.ndimage

2024-12-31 12:11:13.088201: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-31 12:11:13.657013: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-31 12:11:13.657121: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-31 12:11:13.761152: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-31 12:11:13.956113: I tensorflow/core/platform/cpu_feature_guar

In [4]:
# Function to load NIfTI files with memory mapping
def load_nifti_memmap(file_path):
    img = nib.load(file_path)
    data = img.get_fdata(dtype=np.float32, caching='unchanged')  # Memory-mapped array
    affine = img.affine
    header = img.header
    return data, affine, header

# Generator function to load data in batches
def data_generator(file_list, data_path, mask_path, batch_size, target_shape=None):
    while True:
        np.random.shuffle(file_list)
        for start in range(0, len(file_list), batch_size):
            end = min(start + batch_size, len(file_list))
            batch_files = file_list[start:end]
            
            X_batch = []
            y_batch = []
            
            for filename in batch_files:
                img_path = os.path.join(data_path, filename)
                corresponding_mask_path = os.path.join(mask_path, filename)
                
                image, _, _ = load_nifti_memmap(img_path)
                mask, _, _ = load_nifti_memmap(corresponding_mask_path)
                
                # Ensure image and mask have the same shape (and possibly resize if needed)
                if target_shape:
                    image = resize_volume(image, target_shape)
                    mask = resize_volume(mask, target_shape)
                
                X_batch.append(image)
                y_batch.append(mask)
            
            X_batch = np.array(X_batch)[..., np.newaxis]  # Adding channel dimension
            y_batch = np.array(y_batch)[..., np.newaxis]  # Adding channel dimension
            
            yield X_batch, y_batch

# Function to resize volumes (if needed)
def resize_volume(img, target_shape):
    current_shape = img.shape
    if current_shape == target_shape:
        return img
    # Example: using scipy for interpolation
    resized_img = scipy.ndimage.zoom(img, (target_shape[0]/current_shape[0], target_shape[1]/current_shape[1], target_shape[2]/current_shape[2]), order=3)
    return resized_img

def resize_image(image, target_shape):
    # Resize the image to match the target shape (height, width)
    return resize(image, target_shape, preserve_range=True, anti_aliasing=True)

def pad_or_crop_volume(volume, target_shape):
    current_shape = volume.shape
    
    # Calculate padding width for each dimension
    pad_width = [(0, max(target_shape[i] - current_shape[i], 0)) for i in range(3)]
    
    # Pad the volume to the target shape
    volume = np.pad(volume, pad_width, mode='constant', constant_values=0)
    
    # Calculate cropping dimensions for each dimension
    crop_start = [(volume.shape[i] - target_shape[i]) // 2 for i in range(3)]
    crop_end = [crop_start[i] + target_shape[i] for i in range(3)]
    
    # Crop the volume to the target shape
    slices = [slice(crop_start[i], crop_end[i]) for i in range(3)]
    volume = volume[slices[0], slices[1], slices[2]]
    
    return volume

def calculate_volume(mask, voxel_volume):
    # Calculate the volume based on the mask
    return np.sum(mask) * voxel_volume
    
def unet_3d(input_shape):
    inputs = Input(input_shape)
    
    # Downsampling
    c1 = Conv3D(32, (3, 3, 3), activation='relu', padding='same')(inputs)
    c1 = Conv3D(32, (3, 3, 3), activation='relu', padding='same')(c1)
    c1 = Conv3D(32, (3, 3, 3), activation='relu', padding='same')(c1)  # Added layer
    c1 = Conv3D(32, (3, 3, 3), activation='relu', padding='same')(c1)  # Added layer
    p1 = MaxPooling3D((2, 2, 2))(c1)
    
    c2 = Conv3D(64, (3, 3, 3), activation='relu', padding='same')(p1)
    c2 = Conv3D(64, (3, 3, 3), activation='relu', padding='same')(c2)
    c2 = Conv3D(64, (3, 3, 3), activation='relu', padding='same')(c2)  # Added layer
    c2 = Conv3D(64, (3, 3, 3), activation='relu', padding='same')(c2)  # Added layer
    p2 = MaxPooling3D((2, 2, 2))(c2)
    
    c3 = Conv3D(128, (3, 3, 3), activation='relu', padding='same')(p2)
    c3 = Conv3D(128, (3, 3, 3), activation='relu', padding='same')(c3)
    c3 = Conv3D(128, (3, 3, 3), activation='relu', padding='same')(c3)  # Added layer
    c3 = Conv3D(128, (3, 3, 3), activation='relu', padding='same')(c3)  # Added layer
    p3 = MaxPooling3D((2, 2, 2))(c3)
    
    c4 = Conv3D(256, (3, 3, 3), activation='relu', padding='same')(p3)
    c4 = Conv3D(256, (3, 3, 3), activation='relu', padding='same')(c4)
    c4 = Conv3D(256, (3, 3, 3), activation='relu', padding='same')(c4)  # Added layer
    c4 = Conv3D(256, (3, 3, 3), activation='relu', padding='same')(c4)  # Added layer
    
    # Upsampling
    u5 = UpSampling3D((2, 2, 2))(c4)
    u5 = concatenate([u5, c3])
    c5 = Conv3D(128, (3, 3, 3), activation='relu', padding='same')(u5)
    c5 = Conv3D(128, (3, 3, 3), activation='relu', padding='same')(c5)
    c5 = Conv3D(128, (3, 3, 3), activation='relu', padding='same')(c5)  # Added layer
    c5 = Conv3D(128, (3, 3, 3), activation='relu', padding='same')(c5)  # Added layer
    
    u6 = UpSampling3D((2, 2, 2))(c5)
    u6 = concatenate([u6, c2])
    c6 = Conv3D(64, (3, 3, 3), activation='relu', padding='same')(u6)
    c6 = Conv3D(64, (3, 3, 3), activation='relu', padding='same')(c6)
    c6 = Conv3D(64, (3, 3, 3), activation='relu', padding='same')(c6)  # Added layer
    c6 = Conv3D(64, (3, 3, 3), activation='relu', padding='same')(c6)  # Added layer
    
    u7 = UpSampling3D((2, 2, 2))(c6)
    u7 = concatenate([u7, c1])
    c7 = Conv3D(32, (3, 3, 3), activation='relu', padding='same')(u7)
    c7 = Conv3D(32, (3, 3, 3), activation='relu', padding='same')(c7)
    c7 = Conv3D(32, (3, 3, 3), activation='relu', padding='same')(c7)  # Added layer
    c7 = Conv3D(32, (3, 3, 3), activation='relu', padding='same')(c7)  # Added layer

    outputs = Conv3D(1, (1, 1, 1), activation='sigmoid')(c7)
    
    return Model(inputs, outputs)

    model = Model(inputs=[inputs], outputs=[outputs])
    return model
    
    model = Model(inputs, outputs)
    #model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy', dice_coefficient, jaccard_index])
    return model

# Dice coefficient
def dice_coefficient(y_true, y_pred):
    y_true_f = tf.keras.backend.flatten(y_true)
    y_pred_f = tf.keras.backend.flatten(y_pred)
    intersection = tf.keras.backend.sum(y_true_f * y_pred_f)
    return (2. * intersection + 1) / (tf.keras.backend.sum(y_true_f) + tf.keras.backend.sum(y_pred_f) + 1)

# Jaccard index
def jaccard_index(y_true, y_pred):
    y_true_f = tf.keras.backend.flatten(y_true)
    y_pred_f = tf.keras.backend.flatten(y_pred)
    intersection = tf.keras.backend.sum(y_true_f * y_pred_f)
    return (intersection + 1) / (tf.keras.backend.sum(y_true_f) + tf.keras.backend.sum(y_pred_f) - intersection + 1)

# Example paths (update these to your dataset paths)
data_path = r'/home/icmr/Documents/DATASET/MSD_LIVER/Portal/Train/imageTr'  # Update with the correct path
mask_path = r'/home/icmr/Documents/DATASET/MSD_LIVER/Portal/Train/LabelTr'  # Update with the correct path

# Verify paths exist
if not os.path.exists(data_path):
    raise FileNotFoundError(f"Data path not found: {data_path}")
if not os.path.exists(mask_path):
    raise FileNotFoundError(f"Mask path not found: {mask_path}")

# List of files
file_list = [filename for filename in os.listdir(data_path) if filename.endswith('.nii.gz')]
#file_list_test = 
# Split the dataset
train_val_files, test_files = train_test_split(file_list, test_size=0.2, random_state=42)
train_files, val_files = train_test_split(train_val_files, test_size=0.25, random_state=42)  # 0.25 * 0.8 = 0.2

# Define and compile the model
input_shape = (128, 128, 64, 1)  # Example input shape, adjust accordingly
model = unet_3d(input_shape)
model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy', dice_coefficient, jaccard_index])
  
# Define callbacks
checkpoint = ModelCheckpoint('unet3d_best_model.h5', save_best_only=True, monitor='val_loss', mode='min')
#early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Define batch size
batch_size = 2

# Create data generators
train_generator = data_generator(train_files, data_path, mask_path, batch_size, target_shape=input_shape[:3])
val_generator = data_generator(val_files, data_path, mask_path, batch_size, target_shape=input_shape[:3])
test_generator = data_generator(test_files, data_path, mask_path, batch_size, target_shape=input_shape[:3])

# Calculate steps per epoch
steps_per_epoch = len(train_files) // batch_size
validation_steps = len(val_files) // batch_size
test_steps = len(test_files) // batch_size

# Train the model
history = model.fit(train_generator, validation_data=val_generator, epochs=200, steps_per_epoch=steps_per_epoch, validation_steps=validation_steps, callbacks=[checkpoint])

# Save the model
model.save('unet3d_model.h5')

# Evaluate the model on the test set
test_loss, test_accuracy, test_dice, test_jaccard = model.evaluate(test_generator, steps=test_steps)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")
print(f"Test Dice Coefficient: {test_dice}")
print(f"Test Jaccard Index: {test_jaccard}")


2024-12-31 12:11:17.262493: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 43460 MB memory:  -> device: 0, name: NVIDIA RTX A6000, pci bus id: 0000:51:00.0, compute capability: 8.6


Epoch 1/200


2024-12-31 12:11:51.001194: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8902
2024-12-31 12:11:51.337597: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-12-31 12:11:51.986431: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-12-31 12:11:59.449342: I external/local_xla/xla/service/service.cc:168] XLA service 0x718367a56670 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-12-31 12:11:59.449391: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA RTX A6000, Compute Capability 8.6
2024-12-31 12:11:59.468365: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1735627319.637748    4031 device_compiler.h:186] 

15/15 [==============================] - ETA: 0s - loss: 45.9651 - accuracy: 0.7721 - dice_coefficient: 0.1430 - jaccard_index: 0.0793 

/home/icmr/myenv_py311/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


15/15 [==============================] - 334s 22s/step - loss: 45.9651 - accuracy: 0.7721 - dice_coefficient: 0.1430 - jaccard_index: 0.0793 - val_loss: 0.1593 - val_accuracy: 0.8238 - val_dice_coefficient: 0.0503 - val_jaccard_index: 0.0262
Epoch 2/200
15/15 [==============================] - 317s 23s/step - loss: 0.1349 - accuracy: 0.8156 - dice_coefficient: 0.1168 - jaccard_index: 0.0641 - val_loss: 0.1072 - val_accuracy: 0.8047 - val_dice_coefficient: 0.2065 - val_jaccard_index: 0.1156
Epoch 3/200
15/15 [==============================] - 265s 19s/step - loss: 0.0928 - accuracy: 0.8191 - dice_coefficient: 0.2255 - jaccard_index: 0.1295 - val_loss: 0.0967 - val_accuracy: 0.8032 - val_dice_coefficient: 0.2711 - val_jaccard_index: 0.1574
Epoch 4/200
15/15 [==============================] - 244s 17s/step - loss: 0.0779 - accuracy: 0.8194 - dice_coefficient: 0.2594 - jaccard_index: 0.1509 - val_loss: 0.0873 - val_accuracy: 0.8101 - val_dice_coefficient: 0.4067 - val_jaccard_index: 0.2578

In [ ]:
# Load necessary libraries
import os
import numpy as np
import nibabel as nib
import scipy.ndimage
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.utils import Sequence
from tensorflow.keras.layers import Input, Conv3D, MaxPooling3D, UpSampling3D, concatenate
import tensorflow as tf

# Load saved model
saved_model_path = 'unet3d_model.h5'
if not os.path.exists(saved_model_path):
    raise FileNotFoundError(f"Saved model file not found: {saved_model_path}")

# Load the pre-trained model
model = load_model(saved_model_path, 
                   custom_objects={'dice_coefficient': dice_coefficient, 'jaccard_index': jaccard_index})

# Freeze initial layers for transfer learning
for layer in model.layers[:len(model.layers) // 2]:
    layer.trainable = False

# Compile the model again after freezing layers
model.compile(optimizer=Adam(learning_rate=1e-4), loss='binary_crossentropy', 
              metrics=['accuracy', dice_coefficient, jaccard_index])

# Example paths (update these to your new dataset paths)
new_data_path = r'/path/to/new/test/data/images'  # Update with the correct path
new_mask_path = r'/path/to/new/test/data/masks'   # Update with the correct path

# Verify paths exist
if not os.path.exists(new_data_path):
    raise FileNotFoundError(f"New data path not found: {new_data_path}")
if not os.path.exists(new_mask_path):
    raise FileNotFoundError(f"New mask path not found: {new_mask_path}")

# List of new files
new_file_list = [filename for filename in os.listdir(new_data_path) if filename.endswith('.nii.gz')]

# Split the dataset for testing
_, test_files = train_test_split(new_file_list, test_size=1.0, random_state=42)

# Define batch size and target shape
batch_size = 2
target_shape = (128, 128, 64)  # Adjust as necessary

def data_generator(file_list, data_path, mask_path, batch_size, target_shape=None):
    while True:
        np.random.shuffle(file_list)
        for start in range(0, len(file_list), batch_size):
            end = min(start + batch_size, len(file_list))
            batch_files = file_list[start:end]

            X_batch = []
            y_batch = []

            for filename in batch_files:
                img_path = os.path.join(data_path, filename)
                corresponding_mask_path = os.path.join(mask_path, filename)

                image, _, _ = load_nifti_memmap(img_path)
                mask, _, _ = load_nifti_memmap(corresponding_mask_path)

                # Resize image and mask if needed
                if target_shape:
                    image = resize_volume(image, target_shape)
                    mask = resize_volume(mask, target_shape)

                X_batch.append(image)
                y_batch.append(mask)

            X_batch = np.array(X_batch)[..., np.newaxis]  # Adding channel dimension
            y_batch = np.array(y_batch)[..., np.newaxis]  # Adding channel dimension

            yield X_batch, y_batch

# Create test data generator
test_generator = data_generator(test_files, new_data_path, new_mask_path, batch_size, target_shape=target_shape)

test_steps = len(test_files) // batch_size

# Evaluate the model on the test set
test_loss, test_accuracy, test_dice, test_jaccard = model.evaluate(test_generator, steps=test_steps)

print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")
print(f"Test Dice Coefficient: {test_dice}")
print(f"Test Jaccard Index: {test_jaccard}")
